# 06 - Invoke the Agent Executor Lambda

In this notebook, we show how you can invoke the AWS Lambda function created by the CDK stack that builds the backend infrastructure. This Lambda function acts as the agent executor, which orchestrates the overall LLM assistant behaviour.

To run this notebook successfully in SageMaker Studio or SageMaker notebooks, follow the steps below:

1. Make sure that you have run the CDK stack in the backend folder to create the AWS Lambda function and the rest of the backend architecture.
2. Copy the IAM policy ARN in the CDK stack output and attach it to your current SageMaker Execution Role, which you can get with the following command.
    ```python
    import sagemaker
    sagemaker.get_execution_role()
    ```

By attaching the IAM policy created by the CDK stack to your SageMaker execution role, you give SageMaker the permissions to:

1. Read the SSM parameters values which hold the AWS Lambda function name.
2. Invoke the lambda function.


In [ ]:
import json
import boto3

# Initialize Boto3 clients for Lambda and SSM
lambda_client = boto3.client("lambda")
ssm_client = boto3.client("ssm")

In [ ]:
lambda_function_name_ssm_parameter = (
    "/AgenticLLMAssistant/AgentExecutorLambdaNameParameter"
)

lambda_function_name = ssm_client.get_parameter(Name=lambda_function_name_ssm_parameter)
lambda_function_name = lambda_function_name["Parameter"]["Value"]

In [ ]:
lambda_function_name

In [ ]:
def call_agent_lambda(user_input, session_id):
    payload = {"user_input": user_input, "session_id": session_id}

    try:
        # Call the Lambda function
        lambda_response = lambda_client.invoke(
            FunctionName=lambda_function_name,
            InvocationType="RequestResponse",  # Use 'Event' for asynchronous invocation
            Payload=json.dumps(payload),
        )

        # Parse the Lambda function response
        lambda_result = lambda_response["Payload"].read().decode("utf-8")
        lambda_result = json.loads(lambda_result)

        return lambda_result["body"]
    except Exception as e:
        print(e)

In [ ]:
session_id = "10"
user_input = "What is an LLM agent?"

In [ ]:
%%time
results = call_agent_lambda(user_input, session_id)

In [ ]:
print(results["response"])